In [1]:
# Import the functions used in this project
import awesome_functions as af
import decode_utils as du
from sklearn.externals import joblib

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
def fitNaiveBayesModel(X, y):
    return MultinomialNB(alpha=0.0).fit(X, y)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")
# df_ratio = pd.read_csv("Feature_matrix/df_new_has_all_the_ratio_201808031807.csv")

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [2]:
df_train_cp = pd.read_csv("Feature_matrix/df_train_cp_201808062200.csv")

In [3]:
# import decode_utils as du
# import awesome_functions as af
from sklearn.metrics import confusion_matrix
from IPython.display import display, Markdown

def getAccuracy(y_true, y_pred, data_length, i):
    """
        y_true : 원래 타겟 컬럼의 데이터를 넣어준다.
        y_pred : 예측한 값을 넣어준다.
        data_length : 예측한 데이터의 총 개수를 넣어준다.
    """
    accuracy = round(np.trace(confusion_matrix(y_true, y_pred))/data_length, 4)
    display(Markdown("##### Accuracy : " + str(accuracy) + ", smoothing : " + str(i)))
    return accuracy

def fitNaiveBayesModel_smoothing(X, y, a = 1.0):
    return MultinomialNB(alpha=a).fit(X, y)

def __combine_dd_fl(df_dd, df_fl):
    col_we_dont_need = ["VisitNumber", "TripType", 'Return', 'Monday',\
                        'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    cols_we_need = [col for col in df_dd.columns if col not in col_we_dont_need]
    cols = [col for col in cols_we_need if col != "VisitNumber"]
    return du.concatDf(df_dd[cols], df_fl)
    
def get_concat_df_for_test_df(df_dd, df_fl):
    return __combine_dd_fl(df_dd, df_fl)
    
def get_df_has_specified_vn_list(vn_list, df=None, df_dd=None, df_fl=None, is_in=True,
                                 is_need_to_make_df = False):
    if is_need_to_make_df:
        df = __combine_dd_fl(df_dd, df_fl)
        print("Concat 완료.")
    if is_in:
        return df[df.VisitNumber.isin(vn_list)]
    return df[~df.VisitNumber.isin(vn_list)]

def test_multinomial(df, early_stop = 5):
    X, y = af.get_df_to_fit(df)
    a = np.arange(0, 0.16, 0.01)
    accuracy_ = 0
    count = 0
    result = []
    alpha = []
    for i in a:
        if count >= early_stop:
            best_score = np.array(result).max()
            idx = result.index(best_score)
            display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
            return
        model_nb = fitNaiveBayesModel_smoothing(X, y, i)
        y_pred = model_nb.predict(X)
        accuracy = getAccuracy(y, y_pred, len(X), i)
        if accuracy_ > accuracy:
            count += 1
        accuracy_ = accuracy
        result.append(accuracy)
        alpha.append(i)
    best_score = np.array(result).max()
    idx = result.index(best_score)
    display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
#     return best_score, alpha[idx]

def cal_average_of_accuracy(li1, li2, score_li):    
    li3 = 96574 - len(li1 + li2)
    weight = [len(li1), len(li2), li3]
    result = 0
    for i, score in enumerate(score_li):
        result += weight[i] * score
    result = result/96574
    print(result)
    return result

In [5]:
def getSpecifiedVisitNumberData(df_train, vn):
    display(df_train[df_train.VisitNumber == vn])
#     display(df_train_dd[df_train_dd.VisitNumber == vn])

In [4]:
df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [196]:
df_test_decoded = du.decodeStuffNeedsToBeDecoded(df_test)

In [134]:
tmp = df_decoded.groupby(["VisitNumber", "DepartmentDescription"]).agg({"ScanCount" : np.sum}).reset_index()

In [135]:
getSpecifiedVisitNumberData(tmp, 829)

,VisitNumber,DepartmentDescription,ScanCount
1017,829,"GIRLS WEAR, 4-6X AND 7-14",-2


In [136]:
tmp.groupby(["VisitNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

,VisitNumber,DepartmentDescription,ScanCount
0,5,FINANCIAL SERVICES,-1
1,7,PERSONAL CARE,1
2,7,SHOES,1
3,8,DAIRY,2
4,8,DSD GROCERY,1
5,8,HOUSEHOLD CHEMICALS/SUPP,1
6,8,MEAT - FRESH & FROZEN,1
7,8,NULL,1
8,8,PAINT AND ACCESSORIES,18
9,8,PETS AND SUPPLIES,4


In [142]:
scan_total = tmp.groupby("VisitNumber").sum()["ScanCount"].reset_index(name='ScanCount')

In [138]:
len(scan_total)

95674

In [144]:
sc_li = scan_total.ScanCount.values

In [124]:
vn

array([     5,      7,      8, ..., 191345, 191346, 191347])

In [125]:
vn_dd = df_train_dd.VisitNumber.unique()

In [126]:
not_in_vn = []
for v in vn_dd:
    if v not in vn:
        not_in_vn.append(v)

In [127]:
not_in_vn

[829,
 928,
 1049,
 1516,
 1739,
 2754,
 2840,
 3533,
 4571,
 4769,
 6283,
 7703,
 7776,
 8772,
 9160,
 9375,
 9727,
 10158,
 10515,
 10842,
 11314,
 11474,
 11612,
 11862,
 13381,
 14136,
 14470,
 14781,
 14783,
 15083,
 17124,
 18127,
 18201,
 18773,
 21742,
 22470,
 23442,
 23999,
 24468,
 24474,
 25935,
 26033,
 26238,
 26702,
 27783,
 27789,
 27791,
 28299,
 28469,
 28794,
 29160,
 30007,
 31473,
 31766,
 33352,
 33370,
 34642,
 34674,
 34796,
 34996,
 35788,
 36730,
 37538,
 37933,
 38667,
 40803,
 40995,
 41493,
 43487,
 43492,
 43806,
 45159,
 45543,
 45832,
 46120,
 46130,
 46271,
 47047,
 47094,
 47301,
 47450,
 47758,
 48201,
 48318,
 48806,
 49965,
 50363,
 50610,
 50623,
 52351,
 52354,
 52747,
 52903,
 53025,
 53140,
 53859,
 54352,
 54429,
 54482,
 54505,
 54517,
 55049,
 55385,
 55592,
 55630,
 57307,
 57512,
 58522,
 58578,
 59023,
 60404,
 61428,
 62610,
 62724,
 62786,
 63493,
 63596,
 63789,
 64067,
 64340,
 64669,
 65698,
 66291,
 66488,
 67493,
 70671,
 70822,
 71

In [29]:
cols_do_not_need = ['VisitNumber', 'Return', 'TripType', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday']

In [30]:
df_train_cp[cols_do_not_need]

,VisitNumber,Return,TripType,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5,1,999,0,0,0,0,1,0,0
1,7,0,30,0,0,0,0,1,0,0
2,8,1,26,0,0,0,0,1,0,0
3,9,0,8,0,0,0,0,1,0,0
4,10,0,8,0,0,0,0,1,0,0
5,11,0,35,0,0,0,0,1,0,0
6,12,0,41,0,0,0,0,1,0,0
7,15,0,21,0,0,0,0,1,0,0
8,17,0,6,0,0,0,0,1,0,0
9,19,0,42,0,0,0,0,1,0,0


In [93]:
try_new_stuff = df_decoded.groupby(["Company", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [94]:
try_new_stuff_1 = try_new_stuff.groupby("Company").size().reset_index(name="Count")

In [95]:
cp_li = try_new_stuff_1[try_new_stuff_1.Count == 1].Company.unique()

In [96]:
len(cp_li)

4806

In [187]:
try_new_stuff = df_decoded.groupby(["FinelineNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [188]:
try_new_stuff_1 = try_new_stuff.groupby("FinelineNumber").size().reset_index(name="Count")

In [189]:
fl_li = try_new_stuff_1[try_new_stuff_1.Count == 1].FinelineNumber.unique()

In [190]:
len(fl_li)

2695

In [197]:
fl_test_li = try_new_stuff = df_decoded.groupby(["FinelineNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index().groupby(["FinelineNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [198]:
fl_test_li = fl_test_li.groupby("FinelineNumber").size().reset_index(name="Count")

In [199]:
fl_test_li = fl_test_li[fl_test_li.Count == 1].FinelineNumber.unique()

In [201]:
fl_li = np.intersect1d(fl_test_li, fl_li)

In [203]:
len(fl_li)

2695

In [ ]:
df_decoded

In [205]:
fl_li = fl_li.astype(float)

In [206]:
fl_li = fl_li.astype(str)

In [212]:
cl_fl = pd.concat([df_train_dd, df_train_fl[fl_li], df_train_cp[cp_li]], axis = 1)

In [213]:
cp_fl = cl_fl

In [214]:
del cl_fl

In [ ]:
new_df[["N_COMPANY", "N_DD", "N_UPC", "Scancount_total"]]

In [216]:
cp_fl_copy = cp_fl.copy()

In [217]:
cp_fl[["N_COMPANY", "N_DD", "N_UPC", "Scancount_total"]] = new_df[["N_COMPANY", "N_DD", "N_UPC", "Scancount_total"]]

In [218]:
cp_fl.head()

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,899407,899539,899675,899792,95,98,N_COMPANY,N_DD,N_UPC,Scancount_total
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,2,2
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,15,7,21,28
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,3,3
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,3,3


In [174]:
df_test_fl.columns

Index(['VisitNumber', 'Return', '0.0', '1.0', '2.0', '3.0', '4.0', '5.0',
       '6.0', '7.0',
       ...
       '1627.0', '-1', '8191.0', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday'],
      dtype='object', length=5363)

In [176]:
fl_list = fl_list.astype(float)

In [178]:
fl_list = fl_list.astype(str)

In [168]:
ex = try_new_stuff[try_new_stuff.FinelineNumber.isin(fl_li)]

In [169]:
sc = ex.groupby("DepartmentDescription").agg({"ScanCount" : np.max}).ScanCount.values

In [170]:
sc

array([  18,   46,   29,   51,  113,  262,   81,  190,  126,  131,   21,
        464, 1464, 1418,  175, 1886, 3185,  103,  124,  432, 1541,   17,
         66, 2526,  111,  124,  111,  237, 1074,   70,  318,  151,  817,
        133,   47,  125,  124,  750,  368,  263,  227,   41,  450,   25,
        134,  952, 1587,  495,    9,  186,   17, 1295,  458,  532,    1,
        294,   26,  418,  117,  229,   64,   94,   76])

In [171]:
fl_list = []
for dd in ex.DepartmentDescription.unique():
    tmp = ex[ex.DepartmentDescription == dd]
    sc_li = list(tmp.ScanCount.values)
    fl_li = tmp.FinelineNumber.values
    sc_max = tmp.ScanCount.max()
    idx = sc_li.index(sc_max)
    fl_list.append(fl_li[idx])

In [172]:
fl_list = np.array(fl_list).astype(str)

In [91]:
new_df = pd.read_csv("Feature_matrix/new_df_dd_and_breif_fl_n_dd_n_fl_n_company_n_upc_201808081841.csv")

In [97]:
new_df = pd.concat([new_df, df_train_cp[cp_li]], axis = 1)

In [26]:
af.saveDataFrameToCsv(new_df, "new_df_dd_fl_cp")

In [145]:
new_df.Scancount_total = sc_li

In [140]:
new_df

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,898999,899053,899157,899274,899407,899539,899675,899792,95,98
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11,35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,12,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,15,21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,17,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,19,42,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
new_df[["N_COMPANY", "N_DD", "N_UPC", "Scancount_total"]]

,N_COMPANY,N_DD,N_UPC,Scancount_total
0,0,0,0,-1
1,2,2,2,2
2,15,7,21,28
3,3,2,3,3
4,3,2,3,3
5,4,3,4,4
6,7,4,7,7
7,3,1,6,9
8,4,2,4,4
9,7,6,6,9


In [61]:
len(cp_li)

881

In [67]:
new_stuff = pd.concat([new_df[cols_do_not_need], new_df[cp_li], new_df[["N_COMPANY", "N_DD", "N_UPC", "Scancount_total"]]], axis = 1)

In [113]:
new_stuff.head()

,0.0,1.0,-1,10.0,100.0,1000.0,1001.0,1002.0,1003.0,1004.0,...,892992,893621,897522,897666,897752,898115,N_COMPANY,N_DD,N_UPC,Scancount_total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,2,2
2,0,0,1,0,0,0,3,0,0,0,...,0,0,0,0,0,0,15,7,21,28
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,3,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,3,3


In [118]:
len(scan_total)

95301

In [117]:
new_stuff["Scancount_total"] = scan_total

ValueError: Length of values does not match length of index

In [72]:
df_train_fl.head()

,VisitNumber,Return,TripType,0.0,1.0,2.0,3.0,4.0,5.0,6.0,...,1627.0,-1,8191.0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5,1,999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,7,0,30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,8,0,26,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,9,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,10,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [160]:
len(fl_li)

2501

In [75]:
fl_li = fl_li.astype(float)

In [77]:
fl_li = fl_li.astype(str)

In [80]:
fl_li = [li if li != "1.1" else "-1" for li in fl_li]

In [81]:
new_stuff = pd.concat([df_train_fl[fl_li], new_stuff], axis = 1)

In [83]:
af.saveDataFrameToCsv(new_stuff, "new_stuff")

In [194]:
new_stuff

,0.0,1.0,-1,10.0,100.0,1000.0,1001.0,1002.0,1003.0,1004.0,...,892992,893621,897522,897666,897752,898115,N_COMPANY,N_DD,N_UPC,Scancount_total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,2,2
2,0,0,1,0,0,0,3,0,0,0,...,0,0,0,0,0,0,15,7,21,28
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,3,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,3,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,3,4,4
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,4,7,7
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,1,6,9
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,2,4,4
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,6,6,9


In [193]:
new_df

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,898999,899053,899157,899274,899407,899539,899675,899792,95,98
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11,35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,12,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,15,21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,17,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,19,42,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
new_df.columns

Index(['VisitNumber', 'TripType', 'Return', '1-HR PHOTO', 'ACCESSORIES',
       'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING',
       ...
       '890104', '890968', '891038', '891040', '892992', '893621', '897522',
       '897666', '897752', '898115'],
      dtype='object', length=1028)

In [57]:
cols_to_remove = [col for col in new_df.columns if col not in cp_li]

In [58]:
len(cols_to_remove)

147

In [358]:
def get_flat_type_user(df_, col = "DepartmentDescription"):
    """
        예:) vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded) 반드시 df는 decoding 함수로 decoded된 것을 넣을 것.
    """
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] < 0, 0, df_count["Sc_sum"])
    df_count = df_count.dropna()
    df_count_total = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Total")

    df_merged = pd.merge(df_count, df_count_total, on="VisitNumber")
    df_merged["P"] = df_merged["Sc_sum"].div(df_merged["Total"])
    
    num_category_li = []
    for i, vn in enumerate(df_merged.VisitNumber.unique()):
        tmp_df = df_merged[df_merged["VisitNumber"] == vn]
        if tmp_df.Total.values[0] == 0:
            num_category_li.append(0)
        else:
            num_of_category = len(tmp_df[col].unique())
            num_category_li.append(num_of_category)
        if i % 5000 == 0:
            print(str(i) + "까지 진행됨.")
    
    return num_category_li

In [294]:
sc_total = get_flat_type_user(df_decoded).Total.values

In [317]:
num_fl = get_flat_type_user(df_decoded, "FinelineNumber")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [310]:
num_dd = get_flat_type_user(df_decoded)

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [314]:
new_df["N_DD"] = num_dd

In [318]:
new_df["N_FL"] = num_fl

In [321]:
num_company = get_flat_type_user(df_decoded, "Company")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [322]:
new_df["N_COMPANY"] = num_company

In [323]:
af.saveDataFrameToCsv(new_df, "new_df_dd_and_breif_fl_n_dd_n_fl_n_company")

In [359]:
num_upc = get_flat_type_user(df_decoded, "Upc")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [ ]:
len(num_upc)

In [363]:
len(upc_vn_li)

95301

In [ ]:
not_in_li

In [353]:
not_in_li = []
dd_vn_li = df_train_dd.VisitNumber.unique()
upc_vn_li = num_upc.VisitNumber.unique()
for idx, vn in enumerate(dd_vn_li):
    if vn not in upc_vn_li:
        not_in_li.append(vn)
    if idx % 5000 == 0:
        print(idx)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [380]:
new_df.set_index("VisitNumber", inplace=True)

In [381]:
new_df.at[upc_vn_li,"N_UPC"] = num_upc

In [386]:
new_df = new_df.reset_index()

In [121]:
getSpecifiedVisitNumberData(df_train, 173182)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
580439,9,173182,Friday,4.893966e+11,1,MENS WEAR,262.0


In [355]:
getSpecifiedVisitNumberData(df_train, 829)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
1901,999,829,Friday,4.719849e+11,-1,"GIRLS WEAR, 4-6X AND 7-14",2710.0
1902,999,829,Friday,4.719849e+11,-1,"GIRLS WEAR, 4-6X AND 7-14",2710.0


In [354]:
not_in_li

[829,
 928,
 1049,
 1516,
 1739,
 2754,
 2840,
 3533,
 4571,
 4769,
 6283,
 7703,
 7776,
 8772,
 9160,
 9375,
 9727,
 10158,
 10515,
 10842,
 11314,
 11474,
 11612,
 11862,
 13381,
 14136,
 14470,
 14781,
 14783,
 15083,
 17124,
 18127,
 18201,
 18773,
 21742,
 22470,
 23442,
 23999,
 24468,
 24474,
 25935,
 26033,
 26238,
 26702,
 27783,
 27789,
 27791,
 28299,
 28469,
 28794,
 29160,
 30007,
 31473,
 31766,
 33352,
 33370,
 34642,
 34674,
 34796,
 34996,
 35788,
 36730,
 37538,
 37933,
 38667,
 40803,
 40995,
 41493,
 43487,
 43492,
 43806,
 45159,
 45543,
 45832,
 46120,
 46130,
 46271,
 47047,
 47094,
 47301,
 47450,
 47758,
 48201,
 48318,
 48806,
 49965,
 50363,
 50610,
 50623,
 52351,
 52354,
 52747,
 52903,
 53025,
 53140,
 53859,
 54352,
 54429,
 54482,
 54505,
 54517,
 55049,
 55385,
 55592,
 55630,
 57307,
 57512,
 58522,
 58578,
 59023,
 60404,
 61428,
 62610,
 62724,
 62786,
 63493,
 63596,
 63789,
 64067,
 64340,
 64669,
 65698,
 66291,
 66488,
 67493,
 70671,
 70822,
 71

In [326]:
len(num_upc)

95301

In [361]:
new_df["N_UPC"] = 0

In [387]:
af.saveDataFrameToCsv(new_df, "new_df_dd_and_breif_fl_n_dd_n_fl_n_company_n_upc")

In [295]:
new_df["Scancount_total"] = sc_total

In [320]:
af.saveDataFrameToCsv(new_df, "new_df_dd_and_breif_fl_n_dd_n_fl")

In [66]:
digging_fl = pd.pivot_table(data=try_new_stuff, index="FinelineNumber", columns="DepartmentDescription", values="ScanCount", fill_value=0)

In [67]:
digging_fl

DepartmentDescription,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
FinelineNumber,,,,,,,,,,,,,,,,,,,,,
0.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.0,0,0,14,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,0,16,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,2
5.0,0,0,12,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,3
6.0,0,0,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7.0,0,0,0,0,0,41,0,2,0,0,...,0,0,0,0,0,0,0,0,0,3
8.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [3]:
# df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [403]:
train_fl_cols = df_train.FinelineNumber.unique()

In [404]:
test_fl_cols = df_test.FinelineNumber.unique()

In [401]:
len(train_fl_cols), len(test_fl_cols)

(5196, 5204)

In [406]:
intersected_li = np.intersect1d(train_fl_cols, test_fl_cols)

In [407]:
len(intersected_li)

5045

In [411]:
intersected_li = intersected_li.astype(str)

In [451]:
cols = ['VisitNumber', 'Return', 'TripType', '-1', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday']
test_cols = ['VisitNumber', 'Return', '-1', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday']

In [418]:
new_fl_df = pd.concat([df_train_fl[cols], df_train_fl[intersected_li]], axis = 1)

In [420]:
cols_do_not_need = ['VisitNumber', 'Return', 'TripType', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday']

In [426]:
cols_dd = [col for col in df_train_dd.columns if col not in cols_do_not_need]

In [425]:
df_train_dd["MENS WEAR"] = df_train_dd["MENS WEAR"] + df_train_dd["MENSWEAR"]
df_train_dd.drop("MENSWEAR", axis = 1, inplace = True)

In [453]:
new_fl_test_df = pd.concat([df_test_fl[test_cols], df_test_fl[intersected_li]], axis = 1)

In [ ]:
new_fl_df = pd.concat([df_train_fl[cols], df_train_fl[intersected_li]], axis = 1)

In [427]:
len(cols_dd)

68

In [429]:
new_fl_df = pd.concat([df_train_dd[cols_dd], new_fl_df], axis = 1)

In [460]:
new_fl_test_df = pd.concat([df_test_dd[cols_dd],new_fl_test_df], axis = 1)

In [430]:
new_fl_df.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,9961.0,9966.0,9967.0,9970.0,9971.0,9974.0,9975.0,9991.0,9997.0,9998.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [180]:
len(cp_li), len(fl_list)

(4806, 63)

In [186]:
af.saveDataFrameToCsv(new_df, "new_df_dd_cp_fl")

In [183]:
new_df.head()

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,898999,899053,899157,899274,899407,899539,899675,899792,95,98
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_X , train_y = af.get_df_to_fit(cp_fl)

In [148]:
from scipy.sparse import csr_matrix

In [100]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder().fit(train_y)
y_labeled = label_enc.transform(train_y)

In [149]:
train_X, test_X, train_y, test_y = train_test_split(train_X, y_labeled, random_state=0)

In [150]:
csr_train = csr_matrix(train_X.values)
csr_test = csr_matrix(test_X.values)

In [ ]:
new_fl_test_df

In [468]:
csr_test_fl = csr_matrix(X_test.values)

In [465]:
X_test = af.get_df_to_fit(new_fl_test_df, True)

In [467]:
X_test.drop("index", axis = 1, inplace=True)

In [469]:
dtest_fl = xgb.DMatrix(csr_test_fl, feature_names = X_test.columns)

In [153]:
if xgboost:
    dtrain = xgb.DMatrix(csr_train, label=train_y, feature_names = train_X.columns)
    dtest = xgb.DMatrix(csr_test, label=test_y, feature_names = train_X.columns)
    num_boost_round = 1000
    params = {'objective': 'multi:softprob', 
              'eval_metric': 'mlogloss',
              'num_class':38,
              'max_depth':3,
              'eta': 0.25}

    evals = [(dtrain, 'train'), (dtest, 'eval')]

    bst = xgb.train(params=params,  
                    dtrain=dtrain, 
                    num_boost_round=num_boost_round, 
                    evals=evals,
                    early_stopping_rounds=10,)

[0]	train-mlogloss:2.10064	eval-mlogloss:2.10766
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:1.8387	eval-mlogloss:1.85017
[2]	train-mlogloss:1.67583	eval-mlogloss:1.69189
[3]	train-mlogloss:1.55334	eval-mlogloss:1.57189
[4]	train-mlogloss:1.45877	eval-mlogloss:1.47972
[5]	train-mlogloss:1.38297	eval-mlogloss:1.40538
[6]	train-mlogloss:1.32172	eval-mlogloss:1.34578
[7]	train-mlogloss:1.27176	eval-mlogloss:1.29794
[8]	train-mlogloss:1.22874	eval-mlogloss:1.25638
[9]	train-mlogloss:1.19232	eval-mlogloss:1.2208
[10]	train-mlogloss:1.16054	eval-mlogloss:1.19042
[11]	train-mlogloss:1.13291	eval-mlogloss:1.16449
[12]	train-mlogloss:1.10902	eval-mlogloss:1.14179
[13]	train-mlogloss:1.08701	eval-mlogloss:1.12053
[14]	train-mlogloss:1.06795	eval-mlogloss:1.10223
[15]	train-mlogloss:1.05094	eval-mlogloss:1.08648
[16]	train-mlogloss:1.03517	eval-mlogloss:1.07162
[17]	train-

[156]	train-mlogloss:0.671617	eval-mlogloss:0.822057
[157]	train-mlogloss:0.670749	eval-mlogloss:0.821697
[158]	train-mlogloss:0.669843	eval-mlogloss:0.821316
[159]	train-mlogloss:0.669075	eval-mlogloss:0.821056
[160]	train-mlogloss:0.66828	eval-mlogloss:0.8208
[161]	train-mlogloss:0.667451	eval-mlogloss:0.820442
[162]	train-mlogloss:0.666624	eval-mlogloss:0.820277
[163]	train-mlogloss:0.665801	eval-mlogloss:0.820003
[164]	train-mlogloss:0.664977	eval-mlogloss:0.819704
[165]	train-mlogloss:0.664073	eval-mlogloss:0.819426
[166]	train-mlogloss:0.663244	eval-mlogloss:0.81916
[167]	train-mlogloss:0.662338	eval-mlogloss:0.818793
[168]	train-mlogloss:0.661529	eval-mlogloss:0.818556
[169]	train-mlogloss:0.660796	eval-mlogloss:0.818396
[170]	train-mlogloss:0.660091	eval-mlogloss:0.818108
[171]	train-mlogloss:0.659324	eval-mlogloss:0.817766
[172]	train-mlogloss:0.658493	eval-mlogloss:0.817482
[173]	train-mlogloss:0.657773	eval-mlogloss:0.817278
[174]	train-mlogloss:0.657045	eval-mlogloss:0.8172

[312]	train-mlogloss:0.579145	eval-mlogloss:0.798943
[313]	train-mlogloss:0.57871	eval-mlogloss:0.798842
[314]	train-mlogloss:0.578337	eval-mlogloss:0.798745
[315]	train-mlogloss:0.577896	eval-mlogloss:0.798667
[316]	train-mlogloss:0.577467	eval-mlogloss:0.798543
[317]	train-mlogloss:0.577042	eval-mlogloss:0.798455
[318]	train-mlogloss:0.576652	eval-mlogloss:0.798404
[319]	train-mlogloss:0.576226	eval-mlogloss:0.798335
[320]	train-mlogloss:0.57577	eval-mlogloss:0.798183
[321]	train-mlogloss:0.575314	eval-mlogloss:0.798122
[322]	train-mlogloss:0.574857	eval-mlogloss:0.797961
[323]	train-mlogloss:0.574412	eval-mlogloss:0.797876
[324]	train-mlogloss:0.574006	eval-mlogloss:0.797854
[325]	train-mlogloss:0.573605	eval-mlogloss:0.797824
[326]	train-mlogloss:0.573201	eval-mlogloss:0.797736
[327]	train-mlogloss:0.572766	eval-mlogloss:0.797656
[328]	train-mlogloss:0.572338	eval-mlogloss:0.797641
[329]	train-mlogloss:0.571917	eval-mlogloss:0.797523
[330]	train-mlogloss:0.571543	eval-mlogloss:0.79

In [112]:
if xgboost:
    dtrain = xgb.DMatrix(csr_train, label=train_y, feature_names = train_X.columns)
    dtest = xgb.DMatrix(csr_test, label=test_y, feature_names = train_X.columns)
    num_boost_round = 500
    params = {'objective': 'multi:softprob', 
              'eval_metric': 'mlogloss',
              'num_class': 38,
              'max_depth': 3,
              'eta': 0.24}

    evals = [(dtrain, 'train'), (dtest, 'eval')]

    bst = xgb.train(params=params,  
                    dtrain=dtrain, 
                    num_boost_round=num_boost_round, 
                    evals=evals,
                    early_stopping_rounds=10,)

[0]	train-mlogloss:2.12992	eval-mlogloss:2.13665
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:1.86991	eval-mlogloss:1.88191
[2]	train-mlogloss:1.70593	eval-mlogloss:1.7222
[3]	train-mlogloss:1.58455	eval-mlogloss:1.60298
[4]	train-mlogloss:1.48762	eval-mlogloss:1.50908
[5]	train-mlogloss:1.41143	eval-mlogloss:1.43418
[6]	train-mlogloss:1.34831	eval-mlogloss:1.37247
[7]	train-mlogloss:1.29714	eval-mlogloss:1.32286
[8]	train-mlogloss:1.25167	eval-mlogloss:1.27899
[9]	train-mlogloss:1.214	eval-mlogloss:1.24273
[10]	train-mlogloss:1.18069	eval-mlogloss:1.21028
[11]	train-mlogloss:1.15142	eval-mlogloss:1.18209
[12]	train-mlogloss:1.12573	eval-mlogloss:1.15812
[13]	train-mlogloss:1.10354	eval-mlogloss:1.13652
[14]	train-mlogloss:1.08383	eval-mlogloss:1.11765
[15]	train-mlogloss:1.06626	eval-mlogloss:1.10091
[16]	train-mlogloss:1.05031	eval-mlogloss:1.08629
[17]	train-m

KeyboardInterrupt: 

In [441]:
num_boost_round = 1000

In [442]:
 params = {'objective': 'multi:softprob', 
              'eval_metric': 'mlogloss',
              'num_class':38, 
#               'max_delta_step': 3, 
              'eta': 0.2}

In [443]:
bst = xgb.train(params=params,  
                    dtrain=dtrain, 
                    num_boost_round=num_boost_round, 
                    evals=evals,
                    early_stopping_rounds=10,)

[0]	train-mlogloss:2.46238	eval-mlogloss:2.48716
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.18065	eval-mlogloss:2.21908
[2]	train-mlogloss:1.98483	eval-mlogloss:2.03384
[3]	train-mlogloss:1.83685	eval-mlogloss:1.89434
[4]	train-mlogloss:1.71812	eval-mlogloss:1.78341
[5]	train-mlogloss:1.62044	eval-mlogloss:1.69293
[6]	train-mlogloss:1.53751	eval-mlogloss:1.61664
[7]	train-mlogloss:1.46708	eval-mlogloss:1.55265
[8]	train-mlogloss:1.4052	eval-mlogloss:1.49656
[9]	train-mlogloss:1.35211	eval-mlogloss:1.44846
[10]	train-mlogloss:1.30493	eval-mlogloss:1.40641
[11]	train-mlogloss:1.26263	eval-mlogloss:1.36921
[12]	train-mlogloss:1.22532	eval-mlogloss:1.33705
[13]	train-mlogloss:1.19081	eval-mlogloss:1.30695
[14]	train-mlogloss:1.16008	eval-mlogloss:1.28123
[15]	train-mlogloss:1.13214	eval-mlogloss:1.2574
[16]	train-mlogloss:1.10641	eval-mlogloss:1.23558
[17]	train-

[156]	train-mlogloss:0.5077	eval-mlogloss:0.848753
[157]	train-mlogloss:0.506425	eval-mlogloss:0.848058
[158]	train-mlogloss:0.505152	eval-mlogloss:0.847502
[159]	train-mlogloss:0.503759	eval-mlogloss:0.846905
[160]	train-mlogloss:0.502335	eval-mlogloss:0.846319
[161]	train-mlogloss:0.500829	eval-mlogloss:0.845556
[162]	train-mlogloss:0.499545	eval-mlogloss:0.845179
[163]	train-mlogloss:0.498197	eval-mlogloss:0.844563
[164]	train-mlogloss:0.49689	eval-mlogloss:0.844082
[165]	train-mlogloss:0.495435	eval-mlogloss:0.843484
[166]	train-mlogloss:0.494262	eval-mlogloss:0.843121
[167]	train-mlogloss:0.493124	eval-mlogloss:0.842748
[168]	train-mlogloss:0.491929	eval-mlogloss:0.842192
[169]	train-mlogloss:0.490824	eval-mlogloss:0.841878
[170]	train-mlogloss:0.489713	eval-mlogloss:0.841516
[171]	train-mlogloss:0.488502	eval-mlogloss:0.840998
[172]	train-mlogloss:0.487388	eval-mlogloss:0.840666
[173]	train-mlogloss:0.486207	eval-mlogloss:0.840218
[174]	train-mlogloss:0.485027	eval-mlogloss:0.839

[312]	train-mlogloss:0.366881	eval-mlogloss:0.802913
[313]	train-mlogloss:0.366164	eval-mlogloss:0.80272
[314]	train-mlogloss:0.365477	eval-mlogloss:0.802522
[315]	train-mlogloss:0.364769	eval-mlogloss:0.80234
[316]	train-mlogloss:0.364181	eval-mlogloss:0.802178
[317]	train-mlogloss:0.363585	eval-mlogloss:0.802047
[318]	train-mlogloss:0.362921	eval-mlogloss:0.801924
[319]	train-mlogloss:0.362347	eval-mlogloss:0.801798
[320]	train-mlogloss:0.361796	eval-mlogloss:0.801719
[321]	train-mlogloss:0.361221	eval-mlogloss:0.801638
[322]	train-mlogloss:0.360509	eval-mlogloss:0.801534
[323]	train-mlogloss:0.359772	eval-mlogloss:0.80141
[324]	train-mlogloss:0.359218	eval-mlogloss:0.801387
[325]	train-mlogloss:0.358649	eval-mlogloss:0.801319
[326]	train-mlogloss:0.358017	eval-mlogloss:0.801158
[327]	train-mlogloss:0.357438	eval-mlogloss:0.800997
[328]	train-mlogloss:0.356873	eval-mlogloss:0.800844
[329]	train-mlogloss:0.356193	eval-mlogloss:0.800747
[330]	train-mlogloss:0.355665	eval-mlogloss:0.800

[468]	train-mlogloss:0.288033	eval-mlogloss:0.792379
[469]	train-mlogloss:0.287669	eval-mlogloss:0.79248
[470]	train-mlogloss:0.287255	eval-mlogloss:0.792541
[471]	train-mlogloss:0.286891	eval-mlogloss:0.792574
[472]	train-mlogloss:0.286485	eval-mlogloss:0.792602
[473]	train-mlogloss:0.286089	eval-mlogloss:0.792541
[474]	train-mlogloss:0.285726	eval-mlogloss:0.792508
[475]	train-mlogloss:0.285377	eval-mlogloss:0.792582
[476]	train-mlogloss:0.284994	eval-mlogloss:0.792605
[477]	train-mlogloss:0.284641	eval-mlogloss:0.792515
[478]	train-mlogloss:0.284268	eval-mlogloss:0.792471
Stopping. Best iteration:
[468]	train-mlogloss:0.288033	eval-mlogloss:0.792379



In [ ]:
l

In [450]:
bst.predict(dtrain)

array([[6.1516538e-08, 7.8237994e-09, 4.7941097e-07, ..., 1.9745365e-05,
        3.0934877e-06, 5.4863666e-04],
       [7.5475878e-01, 3.3340933e-07, 4.5371526e-05, ..., 2.4361517e-04,
        1.9073808e-05, 2.4144946e-01],
       [2.0865602e-05, 2.2822618e-07, 9.1180496e-05, ..., 3.7704455e-04,
        2.2888424e-05, 3.8959056e-02],
       ...,
       [3.7450855e-06, 3.5674580e-07, 7.9091835e-05, ..., 9.8725408e-04,
        5.0731958e-04, 2.0700897e-04],
       [4.5191926e-09, 1.6990510e-08, 6.5175473e-06, ..., 1.3291796e-04,
        8.1698946e-04, 8.6807822e-06],
       [1.1618909e-05, 6.0778111e-06, 2.3029534e-04, ..., 6.6929293e-04,
        5.3264768e-05, 1.5684430e-02]], dtype=float32)

In [474]:
%time dtest_fl

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [470]:
y_pred_proba = bst.predict(dtest_fl)

KeyboardInterrupt: 